# Quantamental Database Explorer

Explore your Parquet-based data storage:
- **Parquet Files** (`data/factors/`) - All historical data (prices, factors, macro)
- **DuckDB** - SQL query interface over Parquet files

**No SQLite** - We use Parquet as the single source of truth for efficient analytics!


In [1]:
# Setup: imports and configuration
import sys
from pathlib import Path

import duckdb
import pandas as pd
from IPython.display import display

# Configure pandas to show all columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

# Resolve paths (works from repo root or notebooks/)
CWD = Path.cwd().resolve()
CANDIDATES_ROOT = [CWD, CWD.parent]

for root in CANDIDATES_ROOT:
    if (root / "config" / "settings.py").exists():
        if str(root) not in sys.path:
            sys.path.insert(0, str(root))
        break

def resolve_first_existing(paths: list[Path], fallback: Path) -> Path:
    for p in paths:
        if p.exists():
            return p
    return fallback

PARQUET_DIR = resolve_first_existing(
    [root / "data" / "factors" for root in CANDIDATES_ROOT],
    CWD / "data" / "factors",
)

DUCKDB_PATH = resolve_first_existing(
    [root / "data" / "factors" / "factors.duckdb" for root in CANDIDATES_ROOT],
    CWD / "data" / "factors" / "factors.duckdb",
)

print(f"📁 Parquet Dir: {PARQUET_DIR}")
print(f"🦆 DuckDB: {DUCKDB_PATH}")
print(f"✅ Parquet dir exists: {PARQUET_DIR.exists()}")
print(f"✅ DuckDB exists: {DUCKDB_PATH.exists()}")


📁 Parquet Dir: /Users/andres/Downloads/Cursor/quant/data/factors
🦆 DuckDB: /Users/andres/Downloads/Cursor/quant/data/factors/factors.duckdb
✅ Parquet dir exists: True
✅ DuckDB exists: True


---
## 1. Parquet Files Overview
### Available Datasets


In [2]:
# List all Parquet files in the factors directory
import os

parquet_files = [f for f in os.listdir(PARQUET_DIR) if f.endswith('.parquet')]

print("Available Parquet Files:")
print("=" * 80)

file_info = []
for file in sorted(parquet_files):
    path = PARQUET_DIR / file
    size_mb = path.stat().st_size / (1024 * 1024)
    file_info.append({
        'File': file,
        'Size (MB)': f"{size_mb:.2f}",
        'Path': str(path)
    })

df_files = pd.DataFrame(file_info)
display(df_files)


Available Parquet Files:


,File,Size (MB),Path
0,factors_all.parquet,184.02,/Users/andres/Downloads/Cursor/quant/data/fact...
1,factors_price.parquet,184.02,/Users/andres/Downloads/Cursor/quant/data/fact...
2,macro.parquet,0.24,/Users/andres/Downloads/Cursor/quant/data/fact...
3,macro_z.parquet,1.01,/Users/andres/Downloads/Cursor/quant/data/fact...
4,prices.parquet,20.33,/Users/andres/Downloads/Cursor/quant/data/fact...


---
## 2. Prices Dataset
### Wide Format: Date × Symbols


In [3]:
# prices.parquet
prices_path = PARQUET_DIR / "prices.parquet"

if prices_path.exists():
    df_prices = pd.read_parquet(prices_path)
    
    print("=" * 80)
    print("prices.parquet - Wide format (date × tickers)")
    print("=" * 80)
    print(f"Shape: {df_prices.shape[0]:,} dates × {df_prices.shape[1]} tickers")
    print(f"Index: {df_prices.index.name}")
    print(f"Date range: {df_prices.index.min()} to {df_prices.index.max()}")
    print(f"\nFirst 10 tickers: {list(df_prices.columns[:10])}")
    print(f"Last 10 tickers: {list(df_prices.columns[-10:])}")
    
    print("\nFirst 5 rows:")
    display(df_prices.head(5))
    
    print("\nLast 5 rows:")
    display(df_prices.tail(5))
else:
    print(f"⚠️  prices.parquet not found at {prices_path}")


prices.parquet - Wide format (date × tickers)
Shape: 25,537 dates × 504 tickers
Index: date
Date range: 1927-12-30 00:00:00-05:00 to 2025-11-17 00:00:00-05:00

First 10 tickers: ['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A']
Last 10 tickers: ['WTW', 'WDAY', 'WYNN', 'XEL', 'XYL', 'YUM', 'ZBRA', 'ZBH', 'ZTS', '^GSPC']

First 5 rows:


,MMM,AOS,ABT,ABBV,ACN,ADBE,AMD,AES,AFL,A,APD,ABNB,AKAM,ALB,ARE,ALGN,ALLE,LNT,ALL,GOOGL,GOOG,MO,AMZN,AMCR,AEE,AEP,AXP,AIG,AMT,AWK,AMP,AME,AMGN,APH,ADI,AON,APA,APO,AAPL,AMAT,APP,APTV,ACGL,ADM,ANET,AJG,AIZ,T,ATO,ADSK,ADP,AZO,AVB,AVY,AXON,BKR,BALL,BAC,BAX,BDX,BRK-B,BBY,TECH,BIIB,BLK,BX,XYZ,BK,BA,BKNG,BSX,BMY,AVGO,BR,BRO,BF-B,BLDR,BG,BXP,CHRW,CDNS,CPT,CPB,COF,CAH,KMX,CCL,CARR,CAT,CBOE,CBRE,CDW,COR,CNC,CNP,CF,CRL,SCHW,CHTR,CVX,CMG,CB,CHD,CI,CINF,CTAS,CSCO,C,CFG,CLX,CME,CMS,KO,CTSH,COIN,CL,CMCSA,CAG,COP,ED,STZ,CEG,COO,CPRT,GLW,CPAY,CTVA,CSGP,COST,CTRA,CRWD,CCI,CSX,CMI,CVS,DHR,DRI,DDOG,DVA,DAY,DECK,DE,DELL,DAL,DVN,DXCM,FANG,DLR,DG,DLTR,D,DPZ,DASH,DOV,DOW,DHI,DTE,DUK,DD,EMN,ETN,EBAY,ECL,EIX,EW,EA,ELV,EME,EMR,ETR,EOG,EPAM,EQT,EFX,EQIX,EQR,ERIE,ESS,EL,EG,EVRG,ES,EXC,EXE,EXPE,EXPD,EXR,XOM,FFIV,FDS,FICO,FAST,FRT,FDX,FIS,FITB,FSLR,FE,FI,F,FTNT,FTV,FOXA,FOX,BEN,FCX,GRMN,IT,GE,GEHC,GEV,GEN,GNRC,GD,GIS,GM,GPC,GILD,GPN,GL,GDDY,GS,HAL,HIG,HAS,HCA,DOC,HSIC,HSY,HPE,HLT,HOLX,HD,HON,HRL,HST,HWM,HPQ,HUBB,HUM,HBAN,HII,IBM,IEX,IDXX,ITW,INCY,IR,PODD,INTC,IBKR,ICE,IFF,IP,IPG,INTU,ISRG,IVZ,INVH,IQV,IRM,JBHT,JBL,JKHY,J,JNJ,JCI,JPM,K,KVUE,KDP,KEY,KEYS,KMB,KIM,KMI,KKR,KLAC,KHC,KR,LHX,LH,LRCX,LW,LVS,LDOS,LEN,LII,LLY,LIN,LYV,LKQ,LMT,L,LOW,LULU,LYB,MTB,MPC,MAR,MMC,MLM,MAS,MA,MTCH,MKC,MCD,MCK,MDT,MRK,META,MET,MTD,MGM,MCHP,MU,MSFT,MAA,MRNA,MHK,MOH,TAP,MDLZ,MPWR,MNST,MCO,MS,MOS,MSI,MSCI,NDAQ,NTAP,NFLX,NEM,NWSA,NWS,NEE,NKE,NI,NDSN,NSC,NTRS,NOC,NCLH,NRG,NUE,NVDA,NVR,NXPI,ORLY,OXY,ODFL,OMC,ON,OKE,ORCL,OTIS,PCAR,PKG,PLTR,PANW,PSKY,PH,PAYX,PAYC,PYPL,PNR,PEP,PFE,PCG,PM,PSX,PNW,PNC,POOL,PPG,PPL,PFG,PG,PGR,PLD,PRU,PEG,PTC,PSA,PHM,PWR,QCOM,DGX,RL,RJF,RTX,O,REG,REGN,RF,RSG,RMD,RVTY,HOOD,ROK,ROL,ROP,ROST,RCL,SPGI,CRM,SBAC,SLB,STX,SRE,NOW,SHW,SPG,SWKS,SJM,SW,SNA,SOLV,SO,LUV,SWK,SBUX,STT,STLD,STE,SYK,SMCI,SYF,SNPS,SYY,TMUS,TROW,TTWO,TPR,TRGP,TGT,TEL,TDY,TER,TSLA,TXN,TPL,TXT,TMO,TJX,TKO,TTD,TSCO,TT,TDG,TRV,TRMB,TFC,TYL,TSN,USB,UBER,UDR,ULTA,UNP,UAL,UPS,URI,UNH,UHS,VLO,VTR,VLTO,VRSN,VRSK,VZ,VRTX,VTRS,VICI,V,VST,VMC,WRB,GWW,WAB,WMT,DIS,WBD,WM,WAT,WEC,WFC,WELL,WST,WDC,WY,WSM,WMB,WTW,WDAY,WYNN,XEL,XYL,YUM,ZBRA,ZBH,ZTS,^GSPC
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1927-12-30 00:00:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Last 5 rows:


,MMM,AOS,ABT,ABBV,ACN,ADBE,AMD,AES,AFL,A,APD,ABNB,AKAM,ALB,ARE,ALGN,ALLE,LNT,ALL,GOOGL,GOOG,MO,AMZN,AMCR,AEE,AEP,AXP,AIG,AMT,AWK,AMP,AME,AMGN,APH,ADI,AON,APA,APO,AAPL,AMAT,APP,APTV,ACGL,ADM,ANET,AJG,AIZ,T,ATO,ADSK,ADP,AZO,AVB,AVY,AXON,BKR,BALL,BAC,BAX,BDX,BRK-B,BBY,TECH,BIIB,BLK,BX,XYZ,BK,BA,BKNG,BSX,BMY,AVGO,BR,BRO,BF-B,BLDR,BG,BXP,CHRW,CDNS,CPT,CPB,COF,CAH,KMX,CCL,CARR,CAT,CBOE,CBRE,CDW,COR,CNC,CNP,CF,CRL,SCHW,CHTR,CVX,CMG,CB,CHD,CI,CINF,CTAS,CSCO,C,CFG,CLX,CME,CMS,KO,CTSH,COIN,CL,CMCSA,CAG,COP,ED,STZ,CEG,COO,CPRT,GLW,CPAY,CTVA,CSGP,COST,CTRA,CRWD,CCI,CSX,CMI,CVS,DHR,DRI,DDOG,DVA,DAY,DECK,DE,DELL,DAL,DVN,DXCM,FANG,DLR,DG,DLTR,D,DPZ,DASH,DOV,DOW,DHI,DTE,DUK,DD,EMN,ETN,EBAY,ECL,EIX,EW,EA,ELV,EME,EMR,ETR,EOG,EPAM,EQT,EFX,EQIX,EQR,ERIE,ESS,EL,EG,EVRG,ES,EXC,EXE,EXPE,EXPD,EXR,XOM,FFIV,FDS,FICO,FAST,FRT,FDX,FIS,FITB,FSLR,FE,FI,F,FTNT,FTV,FOXA,FOX,BEN,FCX,GRMN,IT,GE,GEHC,GEV,GEN,GNRC,GD,GIS,GM,GPC,GILD,GPN,GL,GDDY,GS,HAL,HIG,HAS,HCA,DOC,HSIC,HSY,HPE,HLT,HOLX,HD,HON,HRL,HST,HWM,HPQ,HUBB,HUM,HBAN,HII,IBM,IEX,IDXX,ITW,INCY,IR,PODD,INTC,IBKR,ICE,IFF,IP,IPG,INTU,ISRG,IVZ,INVH,IQV,IRM,JBHT,JBL,JKHY,J,JNJ,JCI,JPM,K,KVUE,KDP,KEY,KEYS,KMB,KIM,KMI,KKR,KLAC,KHC,KR,LHX,LH,LRCX,LW,LVS,LDOS,LEN,LII,LLY,LIN,LYV,LKQ,LMT,L,LOW,LULU,LYB,MTB,MPC,MAR,MMC,MLM,MAS,MA,MTCH,MKC,MCD,MCK,MDT,MRK,META,MET,MTD,MGM,MCHP,MU,MSFT,MAA,MRNA,MHK,MOH,TAP,MDLZ,MPWR,MNST,MCO,MS,MOS,MSI,MSCI,NDAQ,NTAP,NFLX,NEM,NWSA,NWS,NEE,NKE,NI,NDSN,NSC,NTRS,NOC,NCLH,NRG,NUE,NVDA,NVR,NXPI,ORLY,OXY,ODFL,OMC,ON,OKE,ORCL,OTIS,PCAR,PKG,PLTR,PANW,PSKY,PH,PAYX,PAYC,PYPL,PNR,PEP,PFE,PCG,PM,PSX,PNW,PNC,POOL,PPG,PPL,PFG,PG,PGR,PLD,PRU,PEG,PTC,PSA,PHM,PWR,QCOM,DGX,RL,RJF,RTX,O,REG,REGN,RF,RSG,RMD,RVTY,HOOD,ROK,ROL,ROP,ROST,RCL,SPGI,CRM,SBAC,SLB,STX,SRE,NOW,SHW,SPG,SWKS,SJM,SW,SNA,SOLV,SO,LUV,SWK,SBUX,STT,STLD,STE,SYK,SMCI,SYF,SNPS,SYY,TMUS,TROW,TTWO,TPR,TRGP,TGT,TEL,TDY,TER,TSLA,TXN,TPL,TXT,TMO,TJX,TKO,TTD,TSCO,TT,TDG,TRV,TRMB,TFC,TYL,TSN,USB,UBER,UDR,ULTA,UNP,UAL,UPS,URI,UNH,UHS,VLO,VTR,VLTO,VRSN,VRSK,VZ,VRTX,VTRS,VICI,V,VST,VMC,WRB,GWW,WAB,WMT,DIS,WBD,WM,WAT,WEC,WFC,WELL,WST,WDC,WY,WSM,WMB,WTW,WDAY,WYNN,XEL,XYL,YUM,ZBRA,ZBH,ZTS,^GSPC
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2025-11-11 00:00:00-05:00,168.669998,65.790001,127.000000,225.169998,242.559998,333.220001,237.520004,14.27,113.790001,149.419998,262.230011,122.529999,90.610001,103.879997,54.849998,144.490005,167.550003,68.019997,207.360001,291.309998,291.739990,58.410000,249.100006,8.46,105.190002,122.730003,370.160004,76.150002,184.789993,130.009995,464.940002,197.639999,338.450012,143.470001,233.410004,349.929993,24.590000,132.880005,275.250000,228.669998,594.909973,81.599998,90.660004,57.630001,134.929993,255.000000,225.520004,25.180000,178.660004,301.859985,255.199997,3756.239990,178.839996,174.309998,572.179993,48.950001,47.250000,53.630001,18.250000,189.020004,497.070007,77.680000,59.180000,159.559998,1085.760010,144.979996,65.910004,111.250000,195.210007,5050.649902,102.500000,48.720001,351.959991,222.869995,80.040001,27.820000,106.910004,96.449997,72.419998,150.699997,318.510010,103.680000,31.549999,220.750000,204.809998,34.139999,26.790001,56.970001,567.929993,259.250000,154.899994,142.669998,365.079987,35.360001,39.689999,83.680000,173.500000,96.120003,209.220001,156.240005,29.809999,292.420013,85.510002,268.000000,166.130005,185.759995,71.709999,100.760002,52.240002,105.029999,278.839996,74.839996,71.610001,73.129997,304.010010,79.470001,27.450001,17.299999,90.690002,99.589996,131.809998,351.670013,71.739998,41.340000,87.930000,290.670013,65.720001,68.410004,913.859

---
## 3. Price Factors
### Momentum, Volatility, Beta


In [4]:
# factors_price.parquet
factors_price_path = PARQUET_DIR / "factors_price.parquet"

if factors_price_path.exists():
    df_factors_price = pd.read_parquet(factors_price_path)
    
    print("=" * 80)
    print("factors_price.parquet - Price-based factors (long format)")
    print("=" * 80)
    print(f"Shape: {df_factors_price.shape[0]:,} rows × {df_factors_price.shape[1]} columns")
    print(f"Index: {df_factors_price.index.names}")
    print(f"Columns: {list(df_factors_price.columns)}")
    
    print("\nFirst 5 rows:")
    display(df_factors_price.head(5))
    
    print("\nLast 5 rows:")
    display(df_factors_price.tail(5))
    
    print("\nDescriptive statistics:")
    display(df_factors_price.describe())
else:
    print(f"⚠️  factors_price.parquet not found at {factors_price_path}")


factors_price.parquet - Price-based factors (long format)
Shape: 12,870,648 rows × 5 columns
Index: ['date', 'symbol']
Columns: ['mom_12_1', 'mom_6_1', 'mom_3_1', 'vol_60d', 'beta_60d']

First 5 rows:


factor                            mom_12_1  mom_6_1  mom_3_1  vol_60d  \
date                      symbol                                        
1927-12-30 00:00:00-05:00 A            NaN      NaN      NaN      NaN   
                          AAPL         NaN      NaN      NaN      NaN   
                          ABBV         NaN      NaN      NaN      NaN   
                          ABNB         NaN      NaN      NaN      NaN   
                          ABT          NaN      NaN      NaN      NaN   

factor                            beta_60d  
date                      symbol            
1927-12-30 00:00:00-05:00 A            NaN  
                          AAPL         NaN  
                          ABBV         NaN  
                          ABNB         NaN  
                          ABT          NaN


Last 5 rows:


factor                            mom_12_1   mom_6_1   mom_3_1   vol_60d  \
date                      symbol                                           
2025-11-17 00:00:00-05:00 YUM     0.043302 -0.001401 -0.027911  0.234343   
                          ZBH    -0.078414  0.087120 -0.031181  0.370411   
                          ZBRA   -0.219862  0.025403 -0.029758  0.420650   
                          ZTS    -0.152483 -0.093939 -0.066810  0.318664   
                          ^GSPC   0.111041  0.148596  0.041991  0.113889   

factor                            beta_60d  
date                      symbol            
2025-11-17 00:00:00-05:00 YUM    -0.137355  
                          ZBH     0.049976  
                          ZBRA    2.152013  
                          ZTS     0.919365  
                          ^GSPC   1.000000


Descriptive statistics:


factor,mom_12_1,mom_6_1,mom_3_1,vol_60d,beta_60d
count,4.521972e+06,4.521972e+06,4.521972e+06,4.502316e+06,4.502316e+06
mean,1.462590e-01,6.480913e-02,2.574794e-02,3.092246e-01,9.291330e-01
std,4.678457e-01,2.688282e-01,1.633385e-01,2.764188e-01,6.820150e-01
min,-1.080000e+01,-1.227273e+01,-3.857143e+00,0.000000e+00,-1.426598e+01
25%,-6.224018e-02,-5.902238e-02,-4.679606e-02,1.948385e-01,5.768246e-01
50%,9.711188e-02,4.863498e-02,2.161559e-02,2.597234e-01,8.998445e-01
75%,2.809518e-01,1.625567e-01,9.157781e-02,3.622310e-01,1.222567e+00
max,5.218276e+01,3.252308e+01,2.539286e+01,5.398988e+01,1.282943e+02


---
## 4. All Factors Combined
### Price + Fundamental Factors


In [5]:
# factors_all.parquet
factors_all_path = PARQUET_DIR / "factors_all.parquet"

if factors_all_path.exists():
    df_factors_all = pd.read_parquet(factors_all_path)
    
    print("=" * 80)
    print("factors_all.parquet - All factors (price + fundamentals)")
    print("=" * 80)
    print(f"Shape: {df_factors_all.shape[0]:,} rows × {df_factors_all.shape[1]} columns")
    print(f"Index: {df_factors_all.index.names}")
    print(f"Columns: {list(df_factors_all.columns)}")
    
    print("\nFirst 5 rows:")
    display(df_factors_all.head(5))
    
    print("\nLast 5 rows:")
    display(df_factors_all.tail(5))
else:
    print(f"⚠️  factors_all.parquet not found at {factors_all_path}")


factors_all.parquet - All factors (price + fundamentals)
Shape: 12,870,648 rows × 5 columns
Index: ['date', 'symbol']
Columns: ['mom_12_1', 'mom_6_1', 'mom_3_1', 'vol_60d', 'beta_60d']

First 5 rows:


factor                            mom_12_1  mom_6_1  mom_3_1  vol_60d  \
date                      symbol                                        
1927-12-30 00:00:00-05:00 A            NaN      NaN      NaN      NaN   
                          AAPL         NaN      NaN      NaN      NaN   
                          ABBV         NaN      NaN      NaN      NaN   
                          ABNB         NaN      NaN      NaN      NaN   
                          ABT          NaN      NaN      NaN      NaN   

factor                            beta_60d  
date                      symbol            
1927-12-30 00:00:00-05:00 A            NaN  
                          AAPL         NaN  
                          ABBV         NaN  
                          ABNB         NaN  
                          ABT          NaN


Last 5 rows:


factor                            mom_12_1   mom_6_1   mom_3_1   vol_60d  \
date                      symbol                                           
2025-11-17 00:00:00-05:00 YUM     0.043302 -0.001401 -0.027911  0.234343   
                          ZBH    -0.078414  0.087120 -0.031181  0.370411   
                          ZBRA   -0.219862  0.025403 -0.029758  0.420650   
                          ZTS    -0.152483 -0.093939 -0.066810  0.318664   
                          ^GSPC   0.111041  0.148596  0.041991  0.113889   

factor                            beta_60d  
date                      symbol            
2025-11-17 00:00:00-05:00 YUM    -0.137355  
                          ZBH     0.049976  
                          ZBRA    2.152013  
                          ZTS     0.919365  
                          ^GSPC   1.000000

---
## 5. Macroeconomic Indicators
### CPI, Unemployment, Fed Funds, Yields


In [6]:
# macro.parquet
macro_path = PARQUET_DIR / "macro.parquet"

if macro_path.exists():
    df_macro = pd.read_parquet(macro_path)
    
    print("=" * 80)
    print("macro.parquet - Macroeconomic indicators")
    print("=" * 80)
    print(f"Shape: {df_macro.shape[0]:,} days × {df_macro.shape[1]} indicators")
    print(f"Index: {df_macro.index.name}")
    print(f"Date range: {df_macro.index.min()} to {df_macro.index.max()}")
    print(f"Columns: {list(df_macro.columns)}")
    
    print("\nFirst 5 rows:")
    display(df_macro.head(5))
    
    print("\nLast 5 rows:")
    display(df_macro.tail(5))
    
    print("\nDescriptive statistics:")
    display(df_macro.describe())
else:
    print(f"⚠️  macro.parquet not found at {macro_path}")


macro.parquet - Macroeconomic indicators
Shape: 20,579 days × 5 indicators
Index: date
Date range: 1947-01-01 00:00:00 to 2025-11-17 00:00:00
Columns: ['cpi_yoy', 'unrate', 'fed_funds', 'dgs10', 't10y2y']

First 5 rows:


,cpi_yoy,unrate,fed_funds,dgs10,t10y2y
date,,,,,
1947-01-01,NaN,NaN,NaN,NaN,NaN
1947-01-02,NaN,NaN,NaN,NaN,NaN
1947-01-03,NaN,NaN,NaN,NaN,NaN
1947-01-06,NaN,NaN,NaN,NaN,NaN
1947-01-07,NaN,NaN,NaN,NaN,NaN



Last 5 rows:


,cpi_yoy,unrate,fed_funds,dgs10,t10y2y
date,,,,,
2025-11-11,0.030227,4.3,4.09,4.13,0.55
2025-11-12,0.030227,4.3,4.09,4.08,0.52
2025-11-13,0.030227,4.3,4.09,4.11,0.53
2025-11-14,0.030227,4.3,4.09,4.14,0.52
2025-11-17,0.030227,4.3,4.09,4.14,0.53



Descriptive statistics:


,cpi_yoy,unrate,fed_funds,dgs10,t10y2y
count,20318.000000,20318.000000,18623.000000,16665.000000,12905.000000
mean,0.035310,5.664913,4.622695,5.826700,0.850166
std,0.028707,1.708899,3.573851,2.943836,0.919278
min,-0.029881,2.500000,0.050000,0.520000,-2.410000
25%,0.016669,4.300000,1.930000,3.880000,0.180000
50%,0.028681,5.500000,4.320000,5.470000,0.800000
75%,0.044118,6.700000,6.170000,7.550000,1.480000
max,0.145892,14.800000,19.100000,15.840000,2.910000


---
## 6. Z-Scored Macro Indicators
### Standardized for Cross-Sectional Analysis


In [7]:
# macro_z.parquet
macro_z_path = PARQUET_DIR / "macro_z.parquet"

if macro_z_path.exists():
    df_macro_z = pd.read_parquet(macro_z_path)
    
    print("=" * 80)
    print("macro_z.parquet - Standardized (z-scored) macro indicators")
    print("=" * 80)
    print(f"Shape: {df_macro_z.shape[0]:,} days × {df_macro_z.shape[1]} indicators")
    print(f"Index: {df_macro_z.index.name}")
    print(f"Date range: {df_macro_z.index.min()} to {df_macro_z.index.max()}")
    print(f"Columns: {list(df_macro_z.columns)}")
    
    print("\nFirst 5 rows:")
    display(df_macro_z.head(5))
    
    print("\nLast 5 rows:")
    display(df_macro_z.tail(5))
    
    print("\nDescriptive statistics (should be ~0 mean, ~1 std):")
    display(df_macro_z.describe())
else:
    print(f"⚠️  macro_z.parquet not found at {macro_z_path}")


macro_z.parquet - Standardized (z-scored) macro indicators
Shape: 20,579 days × 5 indicators
Index: date
Date range: 1947-01-01 00:00:00 to 2025-11-17 00:00:00
Columns: ['macro_z_cpi_yoy', 'macro_z_unrate', 'macro_z_fed_funds', 'macro_z_dgs10', 'macro_z_t10y2y']

First 5 rows:


,macro_z_cpi_yoy,macro_z_unrate,macro_z_fed_funds,macro_z_dgs10,macro_z_t10y2y
date,,,,,
1947-01-01,NaN,NaN,NaN,NaN,NaN
1947-01-02,NaN,NaN,NaN,NaN,NaN
1947-01-03,NaN,NaN,NaN,NaN,NaN
1947-01-06,NaN,NaN,NaN,NaN,NaN
1947-01-07,NaN,NaN,NaN,NaN,NaN



Last 5 rows:


,macro_z_cpi_yoy,macro_z_unrate,macro_z_fed_funds,macro_z_dgs10,macro_z_t10y2y
date,,,,,
2025-11-11,-0.730755,0.181878,0.405059,0.665360,0.592005
2025-11-12,-0.731889,0.184760,0.403894,0.620589,0.548390
2025-11-13,-0.733025,0.187667,0.402729,0.645577,0.564006
2025-11-14,-0.734164,0.190598,0.401564,0.670614,0.550004
2025-11-17,-0.735305,0.193554,0.400397,0.669435,0.565616



Descriptive statistics (should be ~0 mean, ~1 std):


,macro_z_cpi_yoy,macro_z_unrate,macro_z_fed_funds,macro_z_dgs10,macro_z_t10y2y
count,20214.000000,20214.000000,18519.000000,16561.000000,12801.000000
mean,0.099916,-0.050464,0.285099,0.062105,-0.216288
std,1.345158,1.485322,1.486967,1.427273,1.360896
min,-3.716056,-2.331360,-4.265090,-4.875493,-3.712167
25%,-0.766620,-1.257274,-0.958129,-0.946079,-1.360038
50%,-0.164965,-0.380350,0.327103,-0.078837,-0.392876
75%,1.061485,0.902627,1.432476,1.185061,0.920854
max,4.621561,16.756748,6.764902,5.303855,4.099746


---
## 7. DuckDB Query Examples
### SQL Interface Over Parquet Files


In [8]:
# Connect to DuckDB and run SQL queries
if DUCKDB_PATH.exists():
    con = duckdb.connect(str(DUCKDB_PATH))
    
    # Recreate views with absolute paths (ensures queries work from any working directory)
    print("🔧 Recreating DuckDB views with absolute paths...")
    views = {
        'prices': PARQUET_DIR / 'prices.parquet',
        'macro': PARQUET_DIR / 'macro.parquet',
        'macro_z': PARQUET_DIR / 'macro_z.parquet',
        'factors_price': PARQUET_DIR / 'factors_price.parquet',
        'factors_all': PARQUET_DIR / 'factors_all.parquet',
    }
    
    for name, path in views.items():
        if path.exists():
            con.execute(f"CREATE OR REPLACE VIEW {name} AS SELECT * FROM read_parquet('{path.as_posix()}')")
            print(f"   ✅ {name}")
        else:
            print(f"   ⚠️  {name} - file not found at {path}")
    print()
    
    print("=" * 80)
    print("DuckDB Query Examples")
    print("=" * 80)
    
    # Example 1: Get AAPL prices for recent dates
    print("\n1. Recent AAPL prices:")
    query1 = """
    SELECT * FROM prices 
    WHERE symbol = 'AAPL' 
    ORDER BY date DESC 
    LIMIT 10
    """
    # Note: prices is in wide format, so this won't work as-is
    # Let's query factors instead
    
    # Example 2: Get factors for AAPL
    print("\n2. Recent AAPL factors:")
    query2 = """
    SELECT * FROM factors_price 
    WHERE symbol = 'AAPL' 
    ORDER BY date DESC 
    LIMIT 10
    """
    try:
        result = con.sql(query2).df()
        display(result)
    except Exception as e:
        print(f"Query failed: {e}")
    
    # Example 3: Top momentum stocks
    print("\n3. Top 10 momentum stocks (most recent date):")
    query3 = """
    WITH latest_date AS (
        SELECT MAX(date) as max_date FROM factors_price
    )
    SELECT symbol, mom_12_1, mom_6_1, mom_3_1
    FROM factors_price
    WHERE date = (SELECT max_date FROM latest_date)
    AND mom_12_1 IS NOT NULL
    ORDER BY mom_12_1 DESC
    LIMIT 10
    """
    try:
        result = con.sql(query3).df()
        display(result)
    except Exception as e:
        print(f"Query failed: {e}")
    
    con.close()
else:
    print(f"⚠️  DuckDB not found at {DUCKDB_PATH}")


🔧 Recreating DuckDB views with absolute paths...
   ✅ prices
   ✅ macro
   ✅ macro_z
   ✅ factors_price
   ✅ factors_all

DuckDB Query Examples

1. Recent AAPL prices:

2. Recent AAPL factors:


,mom_12_1,mom_6_1,mom_3_1,vol_60d,beta_60d,date,symbol
0,0.078338,0.309564,0.123270,0.229707,0.985631,2025-11-16 23:00:00-06:00,AAPL
1,0.058668,0.251982,0.080646,0.225815,0.948928,2025-11-13 23:00:00-06:00,AAPL
2,0.066503,0.255946,0.087475,0.226414,0.928636,2025-11-12 23:00:00-06:00,AAPL
3,0.070621,0.218280,0.077112,0.226795,1.002980,2025-11-11 23:00:00-06:00,AAPL
4,0.086013,0.206971,0.071044,0.230825,1.025211,2025-11-10 23:00:00-06:00,AAPL
5,0.080518,0.176844,0.056213,0.227729,1.012524,2025-11-09 23:00:00-06:00,AAPL
6,0.115556,0.212860,0.112238,0.228019,1.087449,2025-11-06 23:00:00-06:00,AAPL
7,0.136373,0.225145,0.142095,0.228060,1.094036,2025-11-05 23:00:00-06:00,AAPL
8,0.131461,0.215420,0.124591,0.228142,1.136669,2025-11-04 23:00:00-06:00,AAPL
9,0.148170,0.229077,0.175279,0.229754,1.152345,2025-11-03 23:00:00-06:00,AAPL



3. Top 10 momentum stocks (most recent date):


,symbol,mom_12_1,mom_6_1,mom_3_1
0,HOOD,2.187517,0.943145,0.208216
1,WDC,1.718651,1.950103,0.860431
2,PLTR,1.631464,0.427508,0.136417
3,STX,1.454173,1.158804,0.490499
4,MU,1.268326,1.395780,0.868720
5,AVGO,1.165111,0.517242,0.195926
6,NEM,1.112140,0.663841,0.302716
7,LRCX,1.018314,0.777100,0.445195
8,WBD,0.940614,1.276529,0.720400
9,TPR,0.799147,0.452621,0.158115


---
## Summary

**Parquet-Based Architecture:**
- **Single Source of Truth**: All data in Parquet files (no SQLite)
- **Wide Format**: prices.parquet (date × symbols)
- **Long Format**: factors (date, symbol, factors)
- **DuckDB**: SQL query layer (no storage, just views)

**Incremental Updates:**
- Run `python scripts/update_daily.py` to fetch new data
- Run `python scripts/add_symbol.py TICKER` to add stocks
- Cron job runs daily at 6 PM automatically


In [9]:
# Summary statistics
print("=" * 80)
print("DATA SUMMARY")
print("=" * 80)

summary = []

for file in ['prices', 'factors_price', 'factors_all', 'macro', 'macro_z']:
    path = PARQUET_DIR / f"{file}.parquet"
    if path.exists():
        df = pd.read_parquet(path)
        size_mb = path.stat().st_size / (1024 * 1024)
        
        summary.append({
            'Dataset': file,
            'Rows': f"{df.shape[0]:,}",
            'Columns': df.shape[1],
            'Size (MB)': f"{size_mb:.2f}",
            'Date Range': f"{df.index.get_level_values(0).min() if isinstance(df.index, pd.MultiIndex) else df.index.min()} to {df.index.get_level_values(0).max() if isinstance(df.index, pd.MultiIndex) else df.index.max()}"
        })

df_summary = pd.DataFrame(summary)
display(df_summary)

print("\n✅ All datasets ready for quantamental research!")
print("📚 See INCREMENTAL_UPDATES.md for architecture details")
print("🔄 Run 'python scripts/update_daily.py' to update data")


DATA SUMMARY


,Dataset,Rows,Columns,Size (MB),Date Range
0,prices,"25,537",504,20.33,1927-12-30 00:00:00-05:00 to 2025-11-17 00:00:...
1,factors_price,"12,870,648",5,184.02,1927-12-30 00:00:00-05:00 to 2025-11-17 00:00:...
2,factors_all,"12,870,648",5,184.02,1927-12-30 00:00:00-05:00 to 2025-11-17 00:00:...
3,macro,"20,579",5,0.24,1947-01-01 00:00:00 to 2025-11-17 00:00:00
4,macro_z,"20,579",5,1.01,1947-01-01 00:00:00 to 2025-11-17 00:00:00



✅ All datasets ready for quantamental research!
📚 See INCREMENTAL_UPDATES.md for architecture details
🔄 Run 'python scripts/update_daily.py' to update data


### `macro.parquet` - Macroeconomic indicators


In [10]:
# macro.parquet
macro_path = PARQUET_DIR / "macro.parquet"

if macro_path.exists():
    df_macro = pd.read_parquet(macro_path)
    
    print("=" * 80)
    print("macro.parquet - Macroeconomic indicators")
    print("=" * 80)
    print(f"Shape: {df_macro.shape[0]:,} days × {df_macro.shape[1]} indicators")
    print(f"Index: {df_macro.index.name}")
    print(f"Date range: {df_macro.index.min()} to {df_macro.index.max()}")
    print(f"Columns: {list(df_macro.columns)}")
    
    print("\nFirst 5 rows:")
    display(df_macro.head(5))
    
    print("\nLast 5 rows:")
    display(df_macro.tail(5))
    
    print("\nDescriptive statistics:")
    display(df_macro.describe())
else:
    print(f"⚠️  macro.parquet not found at {macro_path}")


macro.parquet - Macroeconomic indicators
Shape: 20,579 days × 5 indicators
Index: date
Date range: 1947-01-01 00:00:00 to 2025-11-17 00:00:00
Columns: ['cpi_yoy', 'unrate', 'fed_funds', 'dgs10', 't10y2y']

First 5 rows:


,cpi_yoy,unrate,fed_funds,dgs10,t10y2y
date,,,,,
1947-01-01,NaN,NaN,NaN,NaN,NaN
1947-01-02,NaN,NaN,NaN,NaN,NaN
1947-01-03,NaN,NaN,NaN,NaN,NaN
1947-01-06,NaN,NaN,NaN,NaN,NaN
1947-01-07,NaN,NaN,NaN,NaN,NaN



Last 5 rows:


,cpi_yoy,unrate,fed_funds,dgs10,t10y2y
date,,,,,
2025-11-11,0.030227,4.3,4.09,4.13,0.55
2025-11-12,0.030227,4.3,4.09,4.08,0.52
2025-11-13,0.030227,4.3,4.09,4.11,0.53
2025-11-14,0.030227,4.3,4.09,4.14,0.52
2025-11-17,0.030227,4.3,4.09,4.14,0.53



Descriptive statistics:


,cpi_yoy,unrate,fed_funds,dgs10,t10y2y
count,20318.000000,20318.000000,18623.000000,16665.000000,12905.000000
mean,0.035310,5.664913,4.622695,5.826700,0.850166
std,0.028707,1.708899,3.573851,2.943836,0.919278
min,-0.029881,2.500000,0.050000,0.520000,-2.410000
25%,0.016669,4.300000,1.930000,3.880000,0.180000
50%,0.028681,5.500000,4.320000,5.470000,0.800000
75%,0.044118,6.700000,6.170000,7.550000,1.480000
max,0.145892,14.800000,19.100000,15.840000,2.910000


### `macro_z.parquet` - Z-scored macro indicators


In [11]:
# macro_z.parquet
macro_z_path = PARQUET_DIR / "macro_z.parquet"

if macro_z_path.exists():
    df_macro_z = pd.read_parquet(macro_z_path)
    
    print("=" * 80)
    print("macro_z.parquet - Standardized (z-scored) macro indicators")
    print("=" * 80)
    print(f"Shape: {df_macro_z.shape[0]:,} days × {df_macro_z.shape[1]} indicators")
    print(f"Index: {df_macro_z.index.name}")
    print(f"Date range: {df_macro_z.index.min()} to {df_macro_z.index.max()}")
    print(f"Columns: {list(df_macro_z.columns)}")
    
    print("\nFirst 5 rows:")
    display(df_macro_z.head(5))
    
    print("\nLast 5 rows:")
    display(df_macro_z.tail(5))
    
    print("\nDescriptive statistics (should be ~0 mean, ~1 std):")
    display(df_macro_z.describe())
else:
    print(f"⚠️  macro_z.parquet not found at {macro_z_path}")


macro_z.parquet - Standardized (z-scored) macro indicators
Shape: 20,579 days × 5 indicators
Index: date
Date range: 1947-01-01 00:00:00 to 2025-11-17 00:00:00
Columns: ['macro_z_cpi_yoy', 'macro_z_unrate', 'macro_z_fed_funds', 'macro_z_dgs10', 'macro_z_t10y2y']

First 5 rows:


,macro_z_cpi_yoy,macro_z_unrate,macro_z_fed_funds,macro_z_dgs10,macro_z_t10y2y
date,,,,,
1947-01-01,NaN,NaN,NaN,NaN,NaN
1947-01-02,NaN,NaN,NaN,NaN,NaN
1947-01-03,NaN,NaN,NaN,NaN,NaN
1947-01-06,NaN,NaN,NaN,NaN,NaN
1947-01-07,NaN,NaN,NaN,NaN,NaN



Last 5 rows:


,macro_z_cpi_yoy,macro_z_unrate,macro_z_fed_funds,macro_z_dgs10,macro_z_t10y2y
date,,,,,
2025-11-11,-0.730755,0.181878,0.405059,0.665360,0.592005
2025-11-12,-0.731889,0.184760,0.403894,0.620589,0.548390
2025-11-13,-0.733025,0.187667,0.402729,0.645577,0.564006
2025-11-14,-0.734164,0.190598,0.401564,0.670614,0.550004
2025-11-17,-0.735305,0.193554,0.400397,0.669435,0.565616



Descriptive statistics (should be ~0 mean, ~1 std):


,macro_z_cpi_yoy,macro_z_unrate,macro_z_fed_funds,macro_z_dgs10,macro_z_t10y2y
count,20214.000000,20214.000000,18519.000000,16561.000000,12801.000000
mean,0.099916,-0.050464,0.285099,0.062105,-0.216288
std,1.345158,1.485322,1.486967,1.427273,1.360896
min,-3.716056,-2.331360,-4.265090,-4.875493,-3.712167
25%,-0.766620,-1.257274,-0.958129,-0.946079,-1.360038
50%,-0.164965,-0.380350,0.327103,-0.078837,-0.392876
75%,1.061485,0.902627,1.432476,1.185061,0.920854
max,4.621561,16.756748,6.764902,5.303855,4.099746


---
## Summary

**Database Architecture:**
- **SQLite** (`stock_data.db`) - Operational cache for recent data
- **Parquet** files - Analytics-optimized historical datasets
  - `prices.parquet` - Wide format (date × tickers)
  - `factors_price.parquet` - Long format (date, symbol, factors)
  - `factors_all.parquet` - Combined price + fundamental factors
  - `macro.parquet` - Raw macroeconomic indicators
  - `macro_z.parquet` - Standardized macro indicators
